In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))
module_path = str(os.getcwd())+'\\out\\'

from env import roroDeck
from agent import SARSA, TDQLearning
from analysis import *
from algorithms import inversionNumber as invNo
from valuation import evaluator as evm
#from valuation import evaluation
#from viz import *

import numpy as np

In [2]:
n_evaluations = 500
time_out = 50_000

Construct a set with **500** (`n_evaluations`) unique stowage plan evaluations.
These evaluations are based on randomly generated stowage plans by the RORO-deck environment.

If the evaluation of a randomly generated stowage plan is equivalent to another evaluation within the set this stowage plan is discarded.
If the this size is not reached within **50_000** (`time_out`) iterations than this procedure is stopped.

In [15]:
stowage_Plans = set()
env = roroDeck.RoRoDeck(True,lanes=10,rows=14)
evaluator = evm.Evaluator(env.vehicle_data, env.grid)
i = 0
while len(stowage_Plans) < n_evaluations and i < time_out:
    done = False
    env.reset()
    while not done:
        state, reward, done, info = env.step(env.action_space_sample())
    evaluation = evaluator.evaluate(env.get_stowage_plan())
    stowage_Plans.add(evaluation)
    i+=1
    if i%500 == 0:
        print(str(i)
              + ' of {}\t unique stowage plan evaluations:\t'.format(time_out)
              + str(len(stowage_Plans)))
if i == time_out:
    print('\n\nWARNING:\tCould not construct {} evaluations.'.format(n_evaluations))
    print('\t\tActual number is {}'.format(len(stowage_Plans)))
    
stowage_Plans = list(stowage_Plans)
stowage_Plans.sort()


500 of 50000	 unique stowage plan evaluations:	195
1000 of 50000	 unique stowage plan evaluations:	257
1500 of 50000	 unique stowage plan evaluations:	290
2000 of 50000	 unique stowage plan evaluations:	317
2500 of 50000	 unique stowage plan evaluations:	333
3000 of 50000	 unique stowage plan evaluations:	351
3500 of 50000	 unique stowage plan evaluations:	361
4000 of 50000	 unique stowage plan evaluations:	374
4500 of 50000	 unique stowage plan evaluations:	377
5000 of 50000	 unique stowage plan evaluations:	385
5500 of 50000	 unique stowage plan evaluations:	397
6000 of 50000	 unique stowage plan evaluations:	400
6500 of 50000	 unique stowage plan evaluations:	406
7000 of 50000	 unique stowage plan evaluations:	411
7500 of 50000	 unique stowage plan evaluations:	415
8000 of 50000	 unique stowage plan evaluations:	420
8500 of 50000	 unique stowage plan evaluations:	427
9000 of 50000	 unique stowage plan evaluations:	430
9500 of 50000	 unique stowage plan evaluations:	434
10000 of 5000

In [16]:
for i in stowage_Plans:
    print(i)

Mandatory Cargo Loaded: 0.6666666666666666
Number of Shifts: 18.0
Space Utilisation: 0.95
Mandatory Cargo Loaded: 0.75
Number of Shifts: 17.0
Space Utilisation: 0.9666666666666667
Mandatory Cargo Loaded: 0.75
Number of Shifts: 17.0
Space Utilisation: 0.975
Mandatory Cargo Loaded: 0.75
Number of Shifts: 16.0
Space Utilisation: 0.975
Mandatory Cargo Loaded: 0.75
Number of Shifts: 13.0
Space Utilisation: 0.9583333333333334
Mandatory Cargo Loaded: 0.75
Number of Shifts: 13.0
Space Utilisation: 0.975
Mandatory Cargo Loaded: 0.75
Number of Shifts: 12.0
Space Utilisation: 0.9583333333333334
Mandatory Cargo Loaded: 0.75
Number of Shifts: 11.0
Space Utilisation: 0.975
Mandatory Cargo Loaded: 0.75
Number of Shifts: 10.0
Space Utilisation: 0.9583333333333334
Mandatory Cargo Loaded: 0.75
Number of Shifts: 7.0
Space Utilisation: 0.975
Mandatory Cargo Loaded: 0.8333333333333334
Number of Shifts: 23.0
Space Utilisation: 0.95
Mandatory Cargo Loaded: 0.8333333333333334
Number of Shifts: 23.0
Space Util

The next step loads a model and prints the parameter on how this model was generated.

*Note: TD-Q-Learning and SARSA agents have an equivalent execution method therefore it does not matter which agent is initalised.*

In [27]:
agent = SARSA.SARSA(env,module_path)
agent.load_model(module_path+"20200505\\1745\\1745_SARSA_L10_R14_Rf1_A5.p")

for info in agent.q_table["ModelParam"]:
    print(str(info)+'\t\t'+str(agent.q_table["ModelParam"][info]))
print(env.vehicle_data)

Algorithm		SARSA
GAMMA		0.999
ALPHA		0.1
Episodes		1500000
EnvLanes:		10
EnvRows		14
VehicleData		[[ 0  1  2  3  4]
 [ 1  2  1  2  2]
 [ 1  1  0  0  1]
 [ 2  3  2  3  2]
 [ 5  5 -1 -1  2]
 [ 0  0  0  0  1]]
[[ 0  1  2  3  4]
 [ 1  2  1  2  2]
 [ 1  1  0  0  1]
 [ 2  3  2  3  2]
 [ 5  5 -1 -1  2]
 [ 0  0  0  0  1]]


In [26]:
env.reset()
agent.execute()
print(env.loading_sequence)
evaluation = evaluator.evaluate(env.get_stowage_plan())
print(evaluation)
stowage_Plans += [evaluation]
#remove duplicates
stowage_Plans = list(dict.fromkeys(stowage_Plans))
stowage_Plans.sort()

Loading Sequence of RORO-Deck (Lanes: 10 Rows: 14)

1. Load Vehicle Type 	 0 	 in Lane: 	 4 	 Row: 	 0 
2. Load Vehicle Type 	 0 	 in Lane: 	 5 	 Row: 	 0 
3. Load Vehicle Type 	 0 	 in Lane: 	 3 	 Row: 	 1 
4. Load Vehicle Type 	 0 	 in Lane: 	 6 	 Row: 	 1 
5. Load Vehicle Type 	 0 	 in Lane: 	 2 	 Row: 	 2 
6. Load Vehicle Type 	 1 	 in Lane: 	 4 	 Row: 	 2 
7. Load Vehicle Type 	 1 	 in Lane: 	 5 	 Row: 	 2 
8. Load Vehicle Type 	 1 	 in Lane: 	 7 	 Row: 	 2 
9. Load Vehicle Type 	 1 	 in Lane: 	 1 	 Row: 	 3 
10. Load Vehicle Type 	 1 	 in Lane: 	 3 	 Row: 	 3 
11. Load Vehicle Type 	 2 	 in Lane: 	 6 	 Row: 	 3 
12. Load Vehicle Type 	 2 	 in Lane: 	 8 	 Row: 	 3 
13. Load Vehicle Type 	 2 	 in Lane: 	 0 	 Row: 	 4 
14. Load Vehicle Type 	 2 	 in Lane: 	 2 	 Row: 	 4 
15. Load Vehicle Type 	 2 	 in Lane: 	 9 	 Row: 	 4 
16. Load Vehicle Type 	 2 	 in Lane: 	 4 	 Row: 	 5 
17. Load Vehicle Type 	 2 	 in Lane: 	 5 	 Row: 	 5 
18. Load Vehicle Type 	 2 	 in Lane: 	 6 	 Row: 	 5 
19.

In [21]:
#at which postion is the stowage plan of the agent. (maximal Performance 100%)
for ix,i in enumerate(stowage_Plans):
    if i == evaluation:
        print(str(ix+1)+". Position of "+str(len(stowage_Plans))+ \
              "\t Performance of model: "+str((ix+1)/(len(stowage_Plans))))
        break

165. Position of 501	 Performance of model: 0.32934131736526945


In [ ]:
capacity = env.rows - env.end_of_lanes - env.minimal_package

In [ ]:
np.sum((capacity/env.minimal_package).astype('int16')) 